<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/[NOTEBOOK_ID])

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions
- No coding experience required
- Join Innovation Community

Transform your AI ideas into reality through hands-on projects and expert mentorship.

[Start Your Journey](https://www.buildfastwithai.com/genai-course)

---

# GPT 5.1 - Testing & Basics

**Created by:** @BuildFastWithAI  
**Model:** OpenAI GPT 5.1  
**Last Updated:** November 2025

This notebook covers the fundamentals of working with GPT 5.1, including:
- Basic setup and hello world examples
- Tool calling capabilities
- Simple agent implementation
- Quick RAG demo
- Performance metrics and cost estimates

## 1. Setup & Installation

In [ ]:
!pip install -q openai langchain langchain-openai faiss-cpu

In [ ]:
from openai import OpenAI
from google.colab import userdata
import time
import json

# Configure API key from Colab secrets
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    client = OpenAI(api_key=OPENAI_API_KEY)
    print("✅ API key configured successfully")
except Exception as e:
    print(f"❌ Error: {e}")
    print("Please add OPENAI_API_KEY to Colab secrets")

## 2. Basic Example - Hello World

In [ ]:
# Simple generation
response = client.chat.completions.create(
    model="gpt-5.1",
    messages=[
        {"role": "user", "content": "Hello! Introduce yourself and explain what makes you unique."}
    ]
)

print(response.choices[0].message.content)

In [ ]:
# Test with parameters
response = client.chat.completions.create(
    model="gpt-5.1",
    messages=[
        {"role": "user", "content": "Write a creative short story about AI in 3 sentences."}
    ],
    temperature=0.9,
    max_tokens=200,
    top_p=0.95
)

print(response.choices[0].message.content)

## 3. Tool Calling

In [ ]:
# Define tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather information for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City name or location"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "Temperature unit"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "Perform arithmetic operations",
            "parameters": {
                "type": "object",
                "properties": {
                    "operation": {
                        "type": "string",
                        "enum": ["add", "subtract", "multiply", "divide"]
                    },
                    "num1": {"type": "number"},
                    "num2": {"type": "number"}
                },
                "required": ["operation", "num1", "num2"]
            }
        }
    }
]

# Function implementations
def get_weather(location, unit="fahrenheit"):
    return {"location": location, "temperature": 72, "condition": "Sunny", "unit": unit}

def calculator(operation, num1, num2):
    ops = {
        "add": num1 + num2,
        "subtract": num1 - num2,
        "multiply": num1 * num2,
        "divide": num1 / num2 if num2 != 0 else "Error"
    }
    return ops.get(operation, "Invalid")

# Test tool calling
messages = [{"role": "user", "content": "What's the weather in Paris and what is 25 * 4?"}]

response = client.chat.completions.create(
    model="gpt-5.1",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

print(response.choices[0].message)

## 4. Simple Agent Implementation

In [ ]:
class SimpleAgent:
    def __init__(self, model="gpt-5.1"):
        self.model = model
        self.client = client
        self.tools = tools
        self.tool_map = {
            "get_weather": get_weather,
            "calculator": calculator
        }
    
    def run(self, query: str, max_iterations: int = 5):
        messages = [{"role": "user", "content": query}]
        
        for i in range(max_iterations):
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                tools=self.tools,
                tool_choice="auto"
            )
            
            message = response.choices[0].message
            
            # Check for tool calls
            if message.tool_calls:
                messages.append(message)
                
                for tool_call in message.tool_calls:
                    func_name = tool_call.function.name
                    func_args = json.loads(tool_call.function.arguments)
                    
                    print(f"🔧 Calling: {func_name}({func_args})")
                    
                    result = self.tool_map[func_name](**func_args)
                    print(f"   Result: {result}\n")
                    
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": json.dumps(result)
                    })
            else:
                print(f"Final Answer: {message.content}")
                return message.content
        
        return "Max iterations reached"

# Test agent
agent = SimpleAgent()
result = agent.run("What's the weather in Tokyo and calculate 100 divided by 4?")

## 5. Quick RAG Demo

In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

# Sample documents
documents = [
    "GPT 5.1 is OpenAI's latest large language model with enhanced reasoning capabilities.",
    "The model supports advanced function calling and structured output generation.",
    "GPT 5.1 offers improved context understanding and can handle complex multi-step tasks.",
    "It features state-of-the-art performance on coding and mathematical reasoning.",
    "The model is optimized for both quality and efficiency in production environments."
]

# Create embeddings and vector store
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
texts = text_splitter.create_documents(documents)

vectorstore = FAISS.from_documents(texts, embeddings)

# Create QA chain
llm = ChatOpenAI(model="gpt-5.1", temperature=0.3)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3})
)

# Query
query = "What are the key features of GPT 5.1?"
response = qa_chain.run(query)
print(f"Question: {query}")
print(f"\nAnswer: {response}")

## 6. Use Cases

### Customer Support Agent

In [ ]:
response = client.chat.completions.create(
    model="gpt-5.1",
    messages=[
        {"role": "system", "content": "You are a helpful customer support agent."},
        {"role": "user", "content": "I can't log into my account. What should I do?"}
    ]
)

print(response.choices[0].message.content)

### Code Assistant

In [ ]:
response = client.chat.completions.create(
    model="gpt-5.1",
    messages=[
        {"role": "user", "content": "Write a Python function to find the nth Fibonacci number with memoization."}
    ]
)

print(response.choices[0].message.content)

### Data Analysis

In [ ]:
data = {
    "sales": [1000, 1500, 1200, 1800, 2000],
    "months": ["Jan", "Feb", "Mar", "Apr", "May"]
}

response = client.chat.completions.create(
    model="gpt-5.1",
    messages=[
        {"role": "user", "content": f"Analyze this sales data and provide insights: {json.dumps(data)}"}
    ]
)

print(response.choices[0].message.content)

## 7. Performance Metrics & Cost Analysis

In [ ]:
# Benchmark
test_prompts = [
    "What is machine learning?",
    "Explain quantum computing in simple terms.",
    "Write a haiku about AI."
]

total_time = 0
total_tokens = 0

for prompt in test_prompts:
    start = time.time()
    response = client.chat.completions.create(
        model="gpt-5.1",
        messages=[{"role": "user", "content": prompt}]
    )
    elapsed = time.time() - start
    total_time += elapsed
    
    tokens = response.usage.total_tokens
    total_tokens += tokens
    
    print(f"Prompt: {prompt[:50]}...")
    print(f"Time: {elapsed:.2f}s | Tokens: {tokens}\n")

print(f"\n📊 Summary:")
print(f"Total Time: {total_time:.2f}s")
print(f"Average Time: {total_time/len(test_prompts):.2f}s")
print(f"Total Tokens: {total_tokens}")
print(f"\n💰 Estimated Cost (at $0.002/1K tokens): ${(total_tokens/1000) * 0.002:.4f}")

## 8. Key Takeaways

✅ **Strengths:**
- Exceptional reasoning capabilities
- Advanced function calling
- Strong performance on complex tasks
- Excellent code generation

📌 **Best Practices:**
- Use Colab secrets for API keys
- Implement error handling
- Monitor token usage and costs
- Test with different temperature settings

🔗 **Resources:**
- [OpenAI Documentation](https://platform.openai.com/docs)
- [API Reference](https://platform.openai.com/docs/api-reference)
- Follow [@BuildFastWithAI](https://twitter.com/BuildFastWithAI) for more tutorials